In [ ]:
!pip install transformers
!pip install datasets
!pip install torch torchvision torchaudio


In [20]:
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from datasets import load_dataset


In [21]:
# Load the SQuAD dataset
dataset = load_dataset("squad")


README.md:   0%|          | 0.00/7.62k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [22]:
# Explore the first example
print(dataset['train'][0])


{'id': '5733be284776f41900661182', 'title': 'University_of_Notre_Dame', 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.', 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?', 'answers': {'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}}


In [23]:
# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
model = AutoModelForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [24]:
# Example context and question
context = dataset['train'][0]['context']
question = dataset['train'][0]['question']

# Tokenize the inputs
inputs = tokenizer(question, context, return_tensors='pt', truncation=True)


In [25]:
# Get model predictions
with torch.no_grad():
    outputs = model(**inputs)

# Get the start and end logits
start_logits = outputs.start_logits
end_logits = outputs.end_logits

# Find the most probable start and end tokens
start_index = torch.argmax(start_logits)
end_index = torch.argmax(end_logits)


In [26]:
# Get the tokens for the answer
answer_tokens = inputs['input_ids'][0][start_index:end_index + 1]
answer = tokenizer.decode(answer_tokens)

print(f"Question: {question}")
print(f"Answer: {answer}")


Question: To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?
Answer: saint bernadette soubirous


In [27]:
# Load the validation dataset
val_dataset = load_dataset("squad", split='validation')

# Evaluate a few examples
for example in val_dataset.select(range(5)):
    context = example['context']
    question = example['question']

    inputs = tokenizer(question, context, return_tensors='pt', truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)

    start_logits = outputs.start_logits
    end_logits = outputs.end_logits
    start_index = torch.argmax(start_logits)
    end_index = torch.argmax(end_logits)
    answer_tokens = inputs['input_ids'][0][start_index:end_index + 1]
    answer = tokenizer.decode(answer_tokens)

    print(f"Question: {question}")
    print(f"Answer: {answer}\n")


Question: Which NFL team represented the AFC at Super Bowl 50?
Answer: denver broncos

Question: Which NFL team represented the NFC at Super Bowl 50?
Answer: carolina panthers

Question: Where did Super Bowl 50 take place?
Answer: levi's stadium in the san francisco bay area at santa clara, california

Question: Which NFL team won Super Bowl 50?
Answer: denver broncos

Question: What color was used to emphasize the 50th anniversary of the Super Bowl?
Answer: gold



In [29]:
# Define your own context and questions
custom_context = """Artificial Intelligence (AI) is the simulation of human intelligence processes by machines, especially computer systems.
These processes include learning (the acquisition of information and rules for using it), reasoning (using rules to reach approximate or definite conclusions),
and self-correction. Specific applications of AI include expert systems, speech recognition, and machine vision."""

custom_question = "What are Artificial Intelligence's processes?"

# Tokenize the custom question and context
inputs = tokenizer(custom_question, custom_context, return_tensors='pt', truncation=True)

# Run the model to get predictions
with torch.no_grad():
    outputs = model(**inputs)

# Get the start and end logits
start_logits = outputs.start_logits
end_logits = outputs.end_logits

# Find the most probable start and end tokens
start_index = torch.argmax(start_logits)
end_index = torch.argmax(end_logits)

# Get the tokens for the answer
answer_tokens = inputs['input_ids'][0][start_index:end_index + 1]
answer = tokenizer.decode(answer_tokens)

print(f"Custom Question: {custom_question}")
print(f"Answer: {answer}")


Custom Question: What are Artificial Intelligence's processes?
Answer: learning ( the acquisition of information and rules for using it ), reasoning ( using rules to reach approximate or definite conclusions ), and self - correction
